## Chapter 18: Using Random Numbers and Probability Models

The last chapter discussed probability, distributions and random numbers as well as pseudorandom numbers.  In this chapter we will use these ideas to build probability models and answer related questions

In [ ]:
using StatsBase, StatsPlots, Random
Random.seed!(1234)

#### 14.1: Flipping Coins

We can flip 100 coins in a pseudorandom way in julia with

In [ ]:
coins = rand(Bool,100)

This is an array of booleans, but you can interpret this as a 0 for a tails and a 1 for a heads.  We can find the mean of this.  

In [ ]:
mean(coins)

Flipping a single coin over and over again is not so interesting, but here's how we can flip 3 coins.  Each row denotes a 3-flip event. This is done 100 times. 

In [ ]:
coins3 = rand(Bool,100,3)

And the following will determine the number of heads flipped by summing along each row: 

In [ ]:
heads_sum = mapslices(sum,coins3;dims=[2])

The following counts the number of heads for 0, 1, 2, 3

In [ ]:
num_heads = counts(heads_sum,0:3)

In [ ]:
bar(0:3,num_heads, legend=false)

Here's a plot of the simulated dice rolls along with the know probability distribution function.  Note: we have turned the number of heads rolled to a fraction so the comparison is equivalent

In [ ]:
sim_dist = num_heads/sum(num_heads)

In [ ]:
groupedbar(0:3,hcat(sim_dist,[1/8,3/8,3/8,1/8]),label=["simulation" "pdf"])

#### 14.2: Rolling a die

Last chapter, we saw how to represent the rolling of a single die a larger number of times. Recall that we can roll a single die 100 times with

In [ ]:
die100 = rand(1:6,100)

And if we want to count the number of times each number is rolled:

In [ ]:
die_count = counts(die100,1:6)

And we can plot this versus the probability distribution function with:

In [ ]:
groupedbar(1:6,hcat(die_count/sum(die_count),[1/6 for i=1:6]),label=["simulation" "pdf"])

### 14.3: Rolling 2 dice

Let's roll 2 dice now and see what happens.  The following will roll 2 dice 10,000 times:

In [ ]:
dice2=rand(1:6,10_000,2)

Sum along the rows:

In [ ]:
sum_dice2 = mapslices(sum,dice2, dims=[2])

In [ ]:
dice_count = counts(sum_dice2,2:12)

In [ ]:
bar(2:12,dice_count/sum(dice_count), xticks=2:2:12, legend=false)

And here's a plot of the simulation versus the pdf:

In [ ]:
groupedbar(2:12,hcat(dice_count/sum(dice_count),[(6-abs(i-7))/36 for i=2:12]), xticks=2:2:12,
    label=["simulation" "pdf"])

We can also ask some interesting questions about probability of dice. 

Here's the probability of getting a 7

In [ ]:
count(x->x==7,sum_dice2)/length(sum_dice2)

In [ ]:
dice_count[6]/sum(dice_count)

Here's the probability of getting a 10 or more:

In [ ]:
count(x->x>=10,sum_dice2)/length(sum_dice2)

#### Exercise

1. Estimate the probability of getting 6 or less
2. Estimate the probability of getting an even number.

##### Estimate the probability that you get a pair of the same numbers.

### 14.4: Other Probability Models

Consider the following problem:

A round table has 7 chairs around it. Mary and her friend Alisha and 5 other people are given seats at the table in a random manner. What is the probability that Mary and Alisha sit next to each other?

This problem can be found by finding the total number of arrangements that they sit next to each other and divide by the total number of arrangement.  Instead, we will use simulation to do this:

### Model of sitting at table

In [ ]:
names = ["Alisha", "Mary", "p1", "p2", "p3", "p4", "p5"]

This function returns a boolean depending on whether or not the friends sit next to each other. 

In [ ]:
function nextToEachOther(names::Array{String,1})
  a = findfirst(name -> name=="Alisha",names)  # this is the position where Alisha sits
  m = findfirst(name -> name=="Mary",names)    # this is the position where Mary sits
  abs(a-m) == 1 || abs(a-m) == length(names)-1  # return true if either the numbers are 1 away or 6 away
end

Let's check some examples:

In [ ]:
nextToEachOther(names)

In [ ]:
nextToEachOther(["Alisha", "Mary", "person1", "person2", "person3", "person4", "person5"])

In [ ]:
nextToEachOther(["Alisha", "person1", "Mary", "person2", "person3", "person4", "person5"])

In [ ]:
nextToEachOther(["Alisha","person1", "person2", "person3", "person4", "person5", "Mary"])

What we are now going to do is to randomly shuffle the the names and see if the arrangement has the friends sitting next to each other.

This shuffles an array:

In [ ]:
shuffle(names)

Here's a block of code that tests the arrangents 10,000 times

In [ ]:
let
  num_sits = 0
  for i=1:10_000
    if nextToEachOther(shuffle(names))
      num_sits += 1
    end
  end
  num_sits/10_000
end

We can do the analysis to determine that analytically the answer is 1/3, so this is quite close

### 14.5: Calculating π using pseudo random numbers

#### Buffon's needle experiment

#### Circle in the Square

1. generate points in the square $[0,1]\times[0,1]$.
2. determine the number of points within the unit circle
3. The fraction of points with the unit circle approximates $\pi/4$

In [ ]:
pts=rand(100,2);

In [ ]:
scatter(pts[:,1],pts[:,2], legend=false, aspect_ratio = :equal, xlim=(0,1))

In [ ]:
plot!(cos,sin,0,pi/2)

If we increase the number of points that we choose randomly, the answer should tend to $\pi/4$.

First, let's find the distance each point is from the origin:

In [ ]:
dist=mapslices(pt->sqrt(pt[1]^2+pt[2]^2),pts;dims=[2])

In [ ]:
inside_circle = count(d -> d<1, dist)

So this shows that 82% of the points are within the circle and we can approximate $\pi$ by multiplying the percent by 4. 

In [ ]:
inside_circle*4/100

#### create a function to do this

In [ ]:
function calcPi(total_points::Integer)
    pts=rand(total_points,2)
    dist=mapslices(pt->pt[1]^2+pt[2]^2,pts;dims=[2])
    4*count(d->d<1,dist)/total_points
end

In [ ]:
calcPi(10_000)

In [ ]:
@time calcPi(10^6)

In [ ]:
@time calcPi(10^7)

not suprisingly, the amount of time it takes was about 10 times as long as the previous, but still only accurate to 2 or 3 decimals. Let's see if we can write a faster function

In [ ]:
function calcPi2(total_points::Integer)

end

In [ ]:
@time calcPi2(10^7)

In [ ]:
@time calcPi2(10^8)

In [ ]:
@time calcPi2(10^9)

How long would it take to calculuate $\pi$ to 100 decimal places using this method?